In [86]:
# !pip install ydata-profiling pandas

In [1]:
# %pip install matplotlib


In [88]:
import os
import logging
import warnings

# отключаем ворнинги
warnings.filterwarnings("ignore")

# отключаем прогресс-бары
os.environ["YDATA_PROFILING_DISABLE_PROGRESS_BAR"] = "1"
os.environ["PROGRESS_BAR_OFF"] = "True"

# отключаем логи библиотек
logging.getLogger("ydata_profiling").setLevel(logging.CRITICAL)
logging.getLogger("pandas_profiling").setLevel(logging.CRITICAL)
logging.getLogger("matplotlib").setLevel(logging.CRITICAL)

%matplotlib inline

In [89]:
import sqlite3
from pathlib import Path

import pandas as pd

try:
    from ydata_profiling import ProfileReport
    HAS_PROFILING = True
except ImportError:
    HAS_PROFILING = False
    print("⚠ ydata_profiling не установлен — профили не будут строиться.")


from pathlib import Path

ROOT = Path.cwd()   # текущая директория ноутбука
DB_PATH = ROOT.parent / "db" / "shop.db"
print("Используем БД:", DB_PATH.resolve())

conn = sqlite3.connect(DB_PATH)


⚠ ydata_profiling не установлен — профили не будут строиться.
Используем БД: /home/recsys1_user01/recipe_recommender/llm_work/recipe_recommender/db/shop.db


In [90]:
def list_tables(connection):
    query = """
    SELECT name
    FROM sqlite_master
    WHERE type='table'
      AND name NOT LIKE 'sqlite_%'
    ORDER BY name
    """
    return pd.read_sql(query, connection)["name"].tolist()

tables = list_tables(conn)
tables


['category',
 'event',
 'order_item',
 'orders',
 'product',
 'recipe',
 'recipe_product',
 'session_info',
 'user']

In [91]:
def load_table(connection, table_name: str) -> pd.DataFrame:
    return pd.read_sql(f"SELECT * FROM {table_name}", connection)

# Пример: посмотрим на product
if "product" in tables:
    df_product = load_table(conn, "product")
    display(df_product.head())


,id,name,image_url,price,unit,weight_value,weight_unit,rating,shelf_life_days,category_id,brand,description,composition,metadata
0,1,"Абрикос мытый, Фрешбар",https://img.vkusvill.ru/pim/images/site/62d3b0...,165.0,шт,300.0,г,0.0,None,0,ВкусВилл,"Медовой спелости абрикосы, тщательно отобранны...",абрикос свежий,"{""category"": ""Фрукты"", ""subcategory"": ""Фруктов..."
1,2,Абрикос Планета Витаминов без косточки заморож...,https://img.vkusvill.ru/pim/images/site/a2549a...,199.0,шт,250.0,г,4.6,None,1,Планета Витаминов,Линейка Гурманики дополняется новым продуктом ...,Абрикос,"{""category"": ""Замороженные ягоды"", ""subcategor..."
2,3,Абрикосы сушеные,https://img.vkusvill.ru/pim/images/site/96ffdd...,328.0,шт,200.0,г,4.8,None,2,ВкусВилл,Сушеные абрикосы способны быстро утолить голод...,абрикосы сушеные без косточки,"{""category"": ""Орехи, чипсы и снеки"", ""subcateg..."
3,5,Аджика,https://img.vkusvill.ru/pim/images/site/site/2...,184.0,шт,200.0,г,4.8,None,3,ВкусВилл,Острая аджика с насыщенным вкусом и густой тек...,"перец красный сладкий свежий, паста томатная (...","{""category"": ""Консервация"", ""subcategory"": ""Ху..."
4,6,Аджика / Соус Фирменный KERAKUR,https://img.vkusvill.ru/pim/images/site/259f77...,335.0,шт,480.0,г,4.8,None,1,Kerakur,"Аджика неострая , подается к мясным и куриным ...","красный перец, томатная паста, красный острый ...","{""category"": ""Овощные консервы"", ""subcategory""..."


In [92]:
reports_dir = Path("table_reports")
reports_dir.mkdir(exist_ok=True)

key_tables = ["user", "product", "orders", "order_item", "event"]

for t in key_tables:
    if t not in tables:
        continue
    print("=" * 60)
    print(f"⭐ Ключевая таблица: {t}")
    print("=" * 60)
    
    df = load_table(conn, t)
    display(df)
    
    if HAS_PROFILING and len(df) > 0:
        profile = ProfileReport(df, title=f"Key table profile: {t}", minimal=True)
        html_path = reports_dir / f"key_{t}_profile.html"
        profile.to_file(html_path)
        print(f"Profile для {t} сохранён в {html_path}\n")


⭐ Ключевая таблица: user


,id,username,password_hash,id_ab_group,name_ab_group,created_at
0,1,151515,a6e639253cd20c3caf6f4f24049ba6ff2d414ba14ab738...,1,llm_recs,2025-11-18 13:26:36
1,2,asnp,a665a45920422f9d417e4867efdc4fb8a04a1f3fff1fa0...,0,ease_popular,2025-11-18 13:26:51


⭐ Ключевая таблица: product


,id,name,image_url,price,unit,weight_value,weight_unit,rating,shelf_life_days,category_id,brand,description,composition,metadata
0,1,"Абрикос мытый, Фрешбар",https://img.vkusvill.ru/pim/images/site/62d3b0...,165.0,шт,300.0,г,0.0,None,0,ВкусВилл,"Медовой спелости абрикосы, тщательно отобранны...",абрикос свежий,"{""category"": ""Фрукты"", ""subcategory"": ""Фруктов..."
1,2,Абрикос Планета Витаминов без косточки заморож...,https://img.vkusvill.ru/pim/images/site/a2549a...,199.0,шт,250.0,г,4.6,None,1,Планета Витаминов,Линейка Гурманики дополняется новым продуктом ...,Абрикос,"{""category"": ""Замороженные ягоды"", ""subcategor..."
2,3,Абрикосы сушеные,https://img.vkusvill.ru/pim/images/site/96ffdd...,328.0,шт,200.0,г,4.8,None,2,ВкусВилл,Сушеные абрикосы способны быстро утолить голод...,абрикосы сушеные без косточки,"{""category"": ""Орехи, чипсы и снеки"", ""subcateg..."
3,5,Аджика,https://img.vkusvill.ru/pim/images/site/site/2...,184.0,шт,200.0,г,4.8,None,3,ВкусВилл,Острая аджика с насыщенным вкусом и густой тек...,"перец красный сладкий свежий, паста томатная (...","{""category"": ""Консервация"", ""subcategory"": ""Ху..."
4,6,Аджика / Соус Фирменный KERAKUR,https://img.vkusvill.ru/pim/images/site/259f77...,335.0,шт,480.0,г,4.8,None,1,Kerakur,"Аджика неострая , подается к мясным и куриным ...","красный перец, томатная паста, красный острый ...","{""category"": ""Овощные консервы"", ""subcategory""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8704,10458,"Зразы ""Сахалинские"" из горбуши с сыром, зам",https://img.vkusvill.ru/pim/images/site/5478ed...,425.0,шт,300.0,г,4.7,None,10,ВкусВилл,"Зразы с горбушей и сыром, которые при обжарке ...","горбуша, сыр полутвердый м.д.ж в сухом вещест...","{""category"": ""Рыба, икра и морепродукты"", ""sub..."
8705,10459,Зубатка пестрая стейк мороженая,https://img.vkusvill.ru/pim/images/site/1a96f9...,920.0,кг,NaN,,4.6,None,10,ВкусВилл,Дикую зубатку вылавливают в Баренцевом море. А...,"Зубатка пестрая, вода питьевая (защитная глазу...","{""category"": ""Рыба, икра и морепродукты"", ""sub..."
8706,10460,Зубатка пестрая стейк зам,https://img.vkusvill.ru/pim/images/site/7ebdd6...,910.0,кг,NaN,,4.9,None,10,ВкусВилл,Замороженные стейки дикой пёстрой зубатки. Мяс...,"зубатка пёстрая, вода питьевая (защитная глазурь)","{""category"": ""Рыба, икра и морепродукты"", ""sub..."
8707,10461,Зубатка пестрая стейк зам,https://img.vkusvill.ru/pim/images/site/9b5aa6...,595.0,шт,600.0,г,4.7,None,10,ВкусВилл,Замороженные стейки дикой пёстрой зубатки. Мяс...,"Зубатка пестрая, вода питьевая (защитная глазу...","{""category"": ""Рыба, икра и морепродукты"", ""sub..."


⭐ Ключевая таблица: orders


,id,user_id,order_time,status,total_price


⭐ Ключевая таблица: order_item


,id,order_id,product_id,quantity,price


⭐ Ключевая таблица: event


,id,user_id,session_id,event_time,event_type,item_id,page_type,source,experiment_key,variant,position,value,request_id,metadata
0,1,2,1fe87b27d12c4f82b361074edafa9e50,2025-11-18 13:27:34,add_to_cart,1,catalog,catalog,None,None,1,None,None,"{""query"": null, ""cart"": {""1"": 1}}"
1,2,2,1fe87b27d12c4f82b361074edafa9e50,2025-11-18 13:27:34,rec_impression,4577,recs_sidebar,recs,main_recs_ab,ease_popular,1,None,6ad0266e2bc9464894db3d168dc630e4,"{""cart"": {""1"": 1}}"
2,3,2,1fe87b27d12c4f82b361074edafa9e50,2025-11-18 13:27:34,rec_impression,10124,recs_sidebar,recs,main_recs_ab,ease_popular,2,None,6ad0266e2bc9464894db3d168dc630e4,"{""cart"": {""1"": 1}}"
3,4,2,1fe87b27d12c4f82b361074edafa9e50,2025-11-18 13:27:34,rec_impression,7186,recs_sidebar,recs,main_recs_ab,ease_popular,3,None,6ad0266e2bc9464894db3d168dc630e4,"{""cart"": {""1"": 1}}"
4,5,2,1fe87b27d12c4f82b361074edafa9e50,2025-11-18 13:27:34,rec_impression,6922,recs_sidebar,recs,main_recs_ab,ease_popular,4,None,6ad0266e2bc9464894db3d168dc630e4,"{""cart"": {""1"": 1}}"
5,6,2,1fe87b27d12c4f82b361074edafa9e50,2025-11-18 13:27:34,rec_impression,4718,recs_sidebar,recs,main_recs_ab,ease_popular,5,None,6ad0266e2bc9464894db3d168dc630e4,"{""cart"": {""1"": 1}}"
6,7,2,1fe87b27d12c4f82b361074edafa9e50,2025-11-18 13:27:34,rec_impression,9524,recs_sidebar,recs,main_recs_ab,ease_popular,6,None,6ad0266e2bc9464894db3d168dc630e4,"{""cart"": {""1"": 1}}"
7,8,2,1fe87b27d12c4f82b361074edafa9e50,2025-11-18 13:27:34,rec_impression,6998,recs_sidebar,recs,main_recs_ab,ease_popular,7,None,6ad0266e2bc9464894db3d168dc630e4,"{""cart"": {""1"": 1}}"
8,9,2,1fe87b27d12c4f82b361074edafa9e50,2025-11-18 13:27:34,rec_impression,10036,recs_sidebar,recs,main_recs_ab,ease_popular,8,None,6ad0266e2bc9464894db3d168dc630e4,"{""cart"": {""1"": 1}}"
9,10,2,1fe87b27d12c4f82b361074edafa9e50,2025-11-18 13:28:16,add_to_cart,2,catalog,catalog,None,None,2,None,None,"{""query"": null, ""cart"": {""2"": 1}}"


In [93]:
def max_matching(adj: dict[int, list[int]]) -> dict[int, int]:

    matchR = {}

    def dfs(u, seen):
        for v in adj[u]:
            if v in seen:
                continue
            seen.add(v)

            if v not in matchR or dfs(matchR[v], seen):
                matchR[v] = u
                return True
        return False

    for u in adj:
        dfs(u, set())

    matchL = {u: v for v, u in matchR.items()}
    return matchL


In [94]:
adj = {
    1:  [2, 5, 7],
    2:  [1, 3],
    3:  [4, 8, 9, 12],
    4:  [5],
    5:  [1, 6, 7, 8],
    6:  [3, 9],
    7:  [2, 4, 6],
    8:  [10, 11],
    9:  [5, 7, 12],
    10: [8]
}



matching = max_matching(adj)
print(len(matching))
print(matching)


10
{7: 2, 2: 1, 3: 4, 4: 5, 6: 3, 5: 6, 1: 7, 8: 10, 10: 8, 9: 12}
